In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')

import pyspark
from pyspark.sql.functions import lit, col, instr, expr, pow, round, bround, corr, count, mean, stddev_pop, min, max
from pyspark.sql.functions import monotonically_increasing_id, initcap, lower, upper, ltrim, rtrim, rpad, lpad, trim
from pyspark.sql.functions import regexp_replace, translate, regexp_extract, current_date, current_timestamp, struct
from pyspark.sql.functions import date_add, date_sub, datediff, months_between, to_date, to_timestamp, coalesce, split, size
from pyspark.sql.functions import array_contains, explode, udf
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, when

from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, FloatType, LongType

from datetime import datetime,timedelta

In [2]:
def get_Spark():

    conf = pyspark.SparkConf().setAll([
        ('spark.submit.deployMode', 'client'), # deploy in yarn-client or yarn-cluster
        ('spark.executor.memory', '8g'),       # memory allocated for each executor
        ('spark.executor.cores', '3'),         # number of cores for each executor
        ('spark.executor.instances', '10'),    # number of executors in total
        ('spark.yarn.am.memory', '10g')])      # memory for spark driver (application master)
    spark = SparkSession.builder \
    .master("yarn") \
    .appName("metric_generation") \
    .enableHiveSupport() \
    .config(conf = conf) \
    .getOrCreate()

    return spark

spark = get_Spark()
spark_context = spark.sparkContext
hc = HiveContext(spark_context)

In [3]:
def hive2spark(hc, query):
    spark_df = hc.sql("""{}""".format(query))
    return spark_df

In [4]:
query = """select vin, start_time, start_day, end_time, \
    from_unixtime(unix_timestamp(CONCAT(start_day, ' ', '07:00:00'), 'yyyyMMdd HH:mm:ss')) as peak_start, \
    from_unixtime(unix_timestamp(CONCAT(start_day, ' ', '09:00:00'), 'yyyyMMdd HH:mm:ss')) as peak_end \
    from ubi.conv_trips_complete"""    

In [5]:
df1 = hive2spark(hc, query)

In [6]:
df1.head()

Row(vin=u'LMGMS1G80H1000254', start_time=u'2018-06-29 06:37:51', start_day=u'20180629', end_time=u'2018-06-29 06:47:22', peak_start=u'2018-06-29 07:00:00', peak_end=u'2018-06-29 09:00:00')

In [7]:
df1.show(5)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 07:00:00|2018-06-29 09:00:00|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 07:00:00|2018-07-07 09:00:00|
|LMGMS1G80J1009719|2018-08-14 02:18:32| 20180814|2018-08-14 02:24:00|2018-08-14 07:00:00|2018-08-14 09:00:00|
|LMGMS1G80J1015701|2019-01-12 01:15:52| 20190112|2019-01-12 01:39:32|2019-01-12 07:00:00|2019-01-12 09:00:00|
|LMGMS1G80J1017061|2019-02-05 01:53:03| 20190205|2019-02-05 02:13:11|2019-02-05 07:00:00|2019-02-05 09:00:00|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
only showi

In [8]:
from pyspark.sql.functions import col, when, expr
from pyspark.sql import functions as F
normaltimeFormat = "yyyyMMddHHmmss"
df1.withColumn("start_time",df1.start_time.astype('Timestamp'))
df1.withColumn("end_time",df1.end_time.astype('Timestamp'))

DataFrame[vin: string, start_time: string, start_day: string, end_time: timestamp, peak_start: string, peak_end: string]

In [9]:
print(type(df1.start_day.astype('date')))

<class 'pyspark.sql.column.Column'>


In [10]:
df1.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 07:00:00|2018-06-29 09:00:00|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 07:00:00|2018-07-07 09:00:00|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
only showing top 2 rows



In [11]:
new_column_1=F.when(df1.start_time > df1.peak_start,df1.start_time.astype('Timestamp'))\
.otherwise(df1.peak_start.astype('Timestamp'))
new_column_2=F.when(df1.end_time < df1.peak_end,df1.end_time.astype('Timestamp'))\
.otherwise(df1.peak_end.astype('Timestamp'))
df1=df1.withColumn("peak_start_time",new_column_1)
df1=df1.withColumn("peak_end_time",new_column_2)

In [12]:
df1.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|    peak_start_time|      peak_end_time|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 07:00:00|2018-06-29 09:00:00|2018-06-29 07:00:00|2018-06-29 06:47:22|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 07:00:00|2018-07-07 09:00:00|2018-07-07 07:00:00|2018-07-07 01:19:43|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
only showing top 2 rows



In [13]:
new_column_3 = (to_timestamp(df1.end_time.astype('Timestamp'),normaltimeFormat).cast("long")
                -to_timestamp(df1.start_time.astype('Timestamp'),normaltimeFormat).cast("long"))/60

In [14]:
df1 = df1.withColumn("driving_mins",new_column_3)
df1.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|    peak_start_time|      peak_end_time|      driving_mins|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 07:00:00|2018-06-29 09:00:00|2018-06-29 07:00:00|2018-06-29 06:47:22| 9.516666666666667|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 07:00:00|2018-07-07 09:00:00|2018-07-07 07:00:00|2018-07-07 01:19:43|19.983333333333334|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------

In [15]:
new_column_4=F.when(df1.peak_end_time > df1.peak_start_time,(to_timestamp(df1.peak_end_time,normaltimeFormat).cast("long")
                                   -to_timestamp(df1.peak_start_time,normaltimeFormat).cast("long"))/60)\
.otherwise(0)
df1=df1.withColumn("peak_driving_mins",new_column_4)

In [16]:
df1.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-----------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|    peak_start_time|      peak_end_time|      driving_mins|peak_driving_mins|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-----------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 07:00:00|2018-06-29 09:00:00|2018-06-29 07:00:00|2018-06-29 06:47:22| 9.516666666666667|              0.0|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 07:00:00|2018-07-07 09:00:00|2018-07-07 07:00:00|2018-07-07 01:19:43|19.983333333333334|              0.0|
+-----------------+-------------------+---------+----------------

In [17]:
print(type(to_date(df1.start_day)))

<class 'pyspark.sql.column.Column'>


In [18]:
agg_df=df1.groupBy("vin","start_day").agg(expr("sum(peak_driving_mins)").alias("daily_peak_driving_mins"),
                                  expr("sum(driving_mins)").alias("daily_driving_mins"))

In [19]:
agg_df=agg_df.withColumn("percent_daily_peak_driving", (F.col("daily_peak_driving_mins") / F.col("daily_driving_mins")))

In [20]:
agg_df.show(2)

+-----------------+---------+-----------------------+------------------+--------------------------+
|              vin|start_day|daily_peak_driving_mins|daily_driving_mins|percent_daily_peak_driving|
+-----------------+---------+-----------------------+------------------+--------------------------+
|LMGMS1G83J1005132| 20180425|                  61.35|243.73333333333335|       0.25170951859956237|
|LMGMS1G85H1S00272| 20180712|     29.316666666666666|417.21666666666664|       0.07026724723365159|
+-----------------+---------+-----------------------+------------------+--------------------------+
only showing top 2 rows



In [21]:
query2 = """select vin, start_day, min(start_time) as start_time from ubi.conv_trips_complete group by vin, start_day"""    

In [22]:
df2 = hive2spark(hc, query2)

In [23]:
df2.show(2)

+-----------------+---------+-------------------+
|              vin|start_day|         start_time|
+-----------------+---------+-------------------+
|LMGMS1G80H1000013| 20180121|2018-01-21 00:21:10|
|LMGMS1G80H1000013| 20181224|2018-12-24 04:49:33|
+-----------------+---------+-------------------+
only showing top 2 rows



In [24]:
#df1=df1.withColumn("thirty_day_after", F.date_add(df1['start_day'].cast("date"), 30))
#months_to_add = 1
#df1 = df1.withColumn("thirty_day_after", F.add_months(df1.start_day.cast("date"), months_to_add))
#from datetime import timedelta
#df1.withColumn("thirty_day_after", expr("date_add("start_day",30)")
#df1=df1.withColumn("thirty_day_after", date_add(to_date(col('start_day').cast('string')),30))
#df1=df1.withColumn("thirty_day_after", datetime.strptime(df1['start_day'].cast("string"),'%Y%m%d'))
#df1=df1.withColumn("thirty_day_after", df1['start_day'].cast('date'))
agg_df=agg_df.withColumn("thirty_day_after", date_add(to_timestamp(df1.start_day,"yyyyMMdd"),30))
#(to_timestamp(df1.end_time.astype('Timestamp'),normaltimeFormat).cast("long")
 #               -to_timestamp(df1.start_time.astype('Timestamp'),normaltimeFormat).cast("long"))/60

In [25]:
agg_df.show(2)
#df1.head()

+-----------------+---------+-----------------------+------------------+--------------------------+----------------+
|              vin|start_day|daily_peak_driving_mins|daily_driving_mins|percent_daily_peak_driving|thirty_day_after|
+-----------------+---------+-----------------------+------------------+--------------------------+----------------+
|LMGMS1G81J1005498| 20180630|     12.533333333333333| 52.88333333333333|       0.23699968484084463|      2018-07-30|
|LMGMS1G89J1014093| 20190309|                    0.0|140.28333333333336|                       0.0|      2019-04-08|
+-----------------+---------+-----------------------+------------------+--------------------------+----------------+
only showing top 2 rows



In [26]:
df = df2.alias("a").join(agg_df.alias("b"), (agg_df.vin == df2.vin) & (df2.start_day <= agg_df.thirty_day_after) & (df2.start_day > agg_df.start_day))\
.select("a.vin","a.start_day","b.daily_peak_driving_mins","b.daily_driving_mins")
df.show(2)

+-----------------+---------+-----------------------+------------------+
|              vin|start_day|daily_peak_driving_mins|daily_driving_mins|
+-----------------+---------+-----------------------+------------------+
|LMGMS1G80J1001278| 20181207|                    0.0| 4.283333333333333|
|LMGMS1G80J1001278| 20181207|     52.516666666666666|110.98333333333332|
+-----------------+---------+-----------------------+------------------+
only showing top 2 rows



In [27]:
res_df=df.groupBy("vin","start_day").agg(expr("sum(daily_peak_driving_mins)").alias("daily_peak_driving_mins"),
                                  expr("sum(daily_driving_mins)").alias("daily_driving_mins"))
res_df=res_df.withColumn("percent_daily_peak_driving", (F.col("daily_peak_driving_mins") / F.col("daily_driving_mins")))

In [28]:
cols = [when(~col(x).isin("NULL", "NA", "NaN",""), col(x)).alias(x) for x in res_df.columns]
res_df = res_df.select(*cols)
res_df.registerTempTable('update_dataframe')
sql_cmd = """CREATE TABLE ubi.temp_am4 AS SELECT vin,start_day, daily_peak_driving_mins, daily_driving_mins,
            percent_daily_peak_driving FROM update_dataframe"""
print(sql_cmd)
hc.sql(sql_cmd)
print('Table temp_am4 creation done.')

CREATE TABLE ubi.temp_am4 AS SELECT vin,start_day, daily_peak_driving_mins, daily_driving_mins,
            percent_daily_peak_driving FROM update_dataframe


AnalysisException: u'`ubi`.`temp_am4` already exists.;'

# Long Distance

In [29]:
query = """select vin, distance, start_day, case
  when distance >= 100 then 1
  else 0
  end as long_distance
 from ubi.conv_trips_complete"""   

In [30]:
df_a = hive2spark(hc, query)

In [31]:
df_a.head()

Row(vin=u'LMGMS1G80H1000254', distance=3.0, start_day=u'20180629', long_distance=0)

In [32]:
df_a.show(2)

+-----------------+--------+---------+-------------+
|              vin|distance|start_day|long_distance|
+-----------------+--------+---------+-------------+
|LMGMS1G80H1000254|     3.0| 20180629|            0|
|LMGMS1G80J1007761|     8.0| 20180707|            0|
+-----------------+--------+---------+-------------+
only showing top 2 rows



In [33]:
df_b = df_a.groupBy("vin","start_day").agg(expr("sum(long_distance)").alias("frequency"),expr("avg(long_distance)").alias("percent_long_distance"))

In [34]:
df_b.head()

Row(vin=u'LMGMS1G83J1006264', start_day=u'20180624', frequency=0, percent_long_distance=0.0)

In [35]:
df_b.show(5)

+-----------------+---------+---------+---------------------+
|              vin|start_day|frequency|percent_long_distance|
+-----------------+---------+---------+---------------------+
|LMGMS1G82J1006787| 20180914|        0|                  0.0|
|LMGMS1G84J1015121| 20181018|        0|                  0.0|
|LMGMS1G84J1020691| 20190407|        0|                  0.0|
|LMGMS1G83J1010864| 20180819|        1| 0.030303030303030304|
|LMGMS1G8XJ1013020| 20190409|        1| 0.047619047619047616|
+-----------------+---------+---------+---------------------+
only showing top 5 rows



In [36]:
df_b = df_b.withColumn("thirty_day_after", date_add(to_timestamp(df_b.start_day,"yyyyMMdd"),30))

In [37]:
df_b.show(5)

+-----------------+---------+---------+---------------------+----------------+
|              vin|start_day|frequency|percent_long_distance|thirty_day_after|
+-----------------+---------+---------+---------------------+----------------+
|LMGMS1G80J1018808| 20181225|        0|                  0.0|      2019-01-24|
|LMGMS1G82J1001752| 20181111|        0|                  0.0|      2018-12-11|
|LMGMS1G87J1006218| 20190301|        0|                  0.0|      2019-03-31|
|LMGMS1G82J1022360| 20190206|        0|                  0.0|      2019-03-08|
|LMGMS1G83J1000982| 20180728|        1|                 0.25|      2018-08-27|
+-----------------+---------+---------+---------------------+----------------+
only showing top 5 rows



In [38]:
df_d = df2.alias("c").join(df_b.alias("b"), (df_b.vin == df2.vin) & (df2.start_day <= df_b.thirty_day_after) & (df2.start_day > df_b.start_day))\
.select("c.vin","c.start_day","b.frequency","b.percent_long_distance") 
df_d.show(5)

+-----------------+---------+---------+---------------------+
|              vin|start_day|frequency|percent_long_distance|
+-----------------+---------+---------+---------------------+
|LMGMS1G80J1001278| 20181225|        0|                  0.0|
|LMGMS1G80J1001278| 20181225|        0|                  0.0|
|LMGMS1G80J1001278| 20181225|        0|                  0.0|
|LMGMS1G80J1001278| 20181225|        0|                  0.0|
|LMGMS1G80J1001278| 20181225|        0|                  0.0|
+-----------------+---------+---------+---------------------+
only showing top 5 rows



In [39]:
df_ld = df_d.groupBy("vin","start_day").agg(expr("sum(frequency)").alias("frequency"),expr("avg(percent_long_distance)").alias("percent_long_distance"))
df_ld.show(5)

+-----------------+---------+---------+---------------------+
|              vin|start_day|frequency|percent_long_distance|
+-----------------+---------+---------+---------------------+
|LMGMS1G80J1001278| 20181207|        0|                  0.0|
|LMGMS1G80J1001278| 20181225|        3|                 0.05|
|LMGMS1G80J1001278| 20181230|        3| 0.039473684210526314|
|LMGMS1G80J1001278| 20181220|        3|  0.06818181818181818|
|LMGMS1G80J1001278| 20181229|        3| 0.041666666666666664|
+-----------------+---------+---------+---------------------+
only showing top 5 rows



In [40]:
cols = [when(~col(x).isin("NULL", "NA", "NaN",""), col(x)).alias(x) for x in df_ld.columns]
df_ld = df_ld.select(*cols)
df_ld.registerTempTable('long_distance')
sql_cmd = """CREATE TABLE ubi.temp_ld AS SELECT vin,start_day, frequency, percent_long_distance FROM long_distance"""
print(sql_cmd)
hc.sql(sql_cmd)
print('Table temp_ld creation done.')

CREATE TABLE ubi.temp_ld AS SELECT vin,start_day, frequency, percent_long_distance FROM long_distance


AnalysisException: u'`ubi`.`temp_ld` already exists.;'

# Night Driving

In [41]:
query = """select vin, start_time, start_day, end_time, 
     from_unixtime(unix_timestamp(CONCAT(start_day, ' ', '22:00:00'), 'yyyyMMdd HH:mm:ss')) as night_start,
   from_unixtime(unix_timestamp(CONCAT(start_day, ' ', '05:00:00'), 'yyyyMMdd HH:mm:ss') + 86400) as night_end from ubi.conv_trips_complete"""    

In [42]:
df_a = hive2spark(hc, query)

In [43]:
df_a.withColumn("start_time",df_a.start_time.astype('Timestamp'))
df_a.withColumn("end_time",df_a.end_time.astype('Timestamp'))

DataFrame[vin: string, start_time: string, start_day: string, end_time: timestamp, night_start: string, night_end: string]

In [44]:
df_a.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
|              vin|         start_time|start_day|           end_time|        night_start|          night_end|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 22:00:00|2018-06-30 05:00:00|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 22:00:00|2018-07-08 05:00:00|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+
only showing top 2 rows



In [45]:
new_column_1=F.when(df_a.start_time > df_a.night_start,df_a.start_time.astype('Timestamp'))\
.otherwise(df_a.night_start.astype('Timestamp'))
new_column_2=F.when(df_a.end_time < df_a.night_end,df_a.end_time.astype('Timestamp'))\
.otherwise(df_a.night_end.astype('Timestamp'))
df_a = df_a.withColumn("night_start_time",new_column_1)
df_b = df_a.withColumn("night_end_time",new_column_2)

In [46]:
df_b.show(5)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
|              vin|         start_time|start_day|           end_time|        night_start|          night_end|   night_start_time|     night_end_time|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 22:00:00|2018-06-30 05:00:00|2018-06-29 22:00:00|2018-06-29 06:47:22|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 22:00:00|2018-07-08 05:00:00|2018-07-07 22:00:00|2018-07-07 01:19:43|
|LMGMS1G80J1009719|2018-08-14 02:18:32| 20180814|2018-08-14 02:24:00|2018-08-14 22:00:00|2018-08-15 05:00:00|2018-08-14 22:00:00|2018-08-14 02:24:00|
|LMGMS1G80J1015701|2019-01-12 01:15:52| 20190112|2019-01-12 01:39:32|2019-01-12 22:00:00|2019-01-13 

In [47]:
new_column_3 = (to_timestamp(df_b.end_time.astype('Timestamp'),normaltimeFormat).cast("long")
                -to_timestamp(df_b.start_time.astype('Timestamp'),normaltimeFormat).cast("long"))/60

In [48]:
df_c = df_b.withColumn("driving_mins",new_column_3)
df_c.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|              vin|         start_time|start_day|           end_time|        night_start|          night_end|   night_start_time|     night_end_time|      driving_mins|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 22:00:00|2018-06-30 05:00:00|2018-06-29 22:00:00|2018-06-29 06:47:22| 9.516666666666667|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 22:00:00|2018-07-08 05:00:00|2018-07-07 22:00:00|2018-07-07 01:19:43|19.983333333333334|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------

In [49]:
new_column_4=F.when(df_c.night_end_time > df_c.night_start_time,(to_timestamp(df_c.night_end_time,normaltimeFormat).cast("long")
                                   -to_timestamp(df_c.night_start_time,normaltimeFormat).cast("long"))/60)\
.otherwise(0)
df_c=df_c.withColumn("night_driving_mins",new_column_4)
df_c.show(3)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
|              vin|         start_time|start_day|           end_time|        night_start|          night_end|   night_start_time|     night_end_time|      driving_mins|night_driving_mins|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 22:00:00|2018-06-30 05:00:00|2018-06-29 22:00:00|2018-06-29 06:47:22| 9.516666666666667|               0.0|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 22:00:00|2018-07-08 05:00:00|2018-07-07 22:00:00|2018-07-07 01:19:43|19.983333333333334|               0.0|
|LMGMS1G80J1009719|2018-08-14 02:18:32| 20180814|2018-08-14 

In [50]:
df_d = df_c.groupBy("vin","start_day").agg(expr("sum(night_driving_mins)").alias("daily_night_driving_mins"),
                                           expr("sum(driving_mins)").alias("daily_driving_mins"))

In [51]:
df_d.show(2)

+-----------------+---------+------------------------+------------------+
|              vin|start_day|daily_night_driving_mins|daily_driving_mins|
+-----------------+---------+------------------------+------------------+
|LMGMS1G86J1011104| 20180821|                     0.0|301.80000000000007|
|LMGMS1G84J1013255| 20180706|                     0.0|             263.7|
+-----------------+---------+------------------------+------------------+
only showing top 2 rows



In [52]:
df_d = df_d.withColumn("thirty_day_after", date_add(to_timestamp(df_c.start_day,"yyyyMMdd"),30))

In [53]:
df_d.show(2)

+-----------------+---------+------------------------+------------------+----------------+
|              vin|start_day|daily_night_driving_mins|daily_driving_mins|thirty_day_after|
+-----------------+---------+------------------------+------------------+----------------+
|LMGMS1G81J1001869| 20190223|                     0.0| 93.11666666666667|      2019-03-25|
|LMGMS1G84J1019234| 20181229|                     0.0|             144.3|      2019-01-28|
+-----------------+---------+------------------------+------------------+----------------+
only showing top 2 rows



In [54]:
df_f = df2.alias("e").join(df_d.alias("d"), (df_d.vin == df2.vin) & (df2.start_day <= df_d.thirty_day_after) & (df2.start_day > df_d.start_day))\
.select("e.vin","e.start_day","d.daily_driving_mins","d.daily_night_driving_mins") 
df_f.show(5)

+-----------------+---------+------------------+------------------------+
|              vin|start_day|daily_driving_mins|daily_night_driving_mins|
+-----------------+---------+------------------+------------------------+
|LMGMS1G80J1001278| 20181209|             209.8|                     0.0|
|LMGMS1G80J1001278| 20181209|110.98333333333332|                     0.0|
|LMGMS1G80J1001278| 20181209| 4.283333333333333|                     0.0|
|LMGMS1G80J1001278| 20181209|237.76666666666668|                     0.0|
|LMGMS1G80J1001278| 20181221|             209.8|                     0.0|
+-----------------+---------+------------------+------------------------+
only showing top 5 rows



In [55]:
df_nd=df_f.groupBy("vin","start_day").agg(expr("sum(daily_night_driving_mins)").alias("daily_night_driving_mins"),
                                  expr("sum(daily_driving_mins)").alias("daily_driving_mins"))
df_nd=df_nd.withColumn("percent_daily_night_driving", (F.col("daily_night_driving_mins") / F.col("daily_driving_mins")))
df_nd.show(2)

+-----------------+---------+------------------------+------------------+---------------------------+
|              vin|start_day|daily_night_driving_mins|daily_driving_mins|percent_daily_night_driving|
+-----------------+---------+------------------------+------------------+---------------------------+
|LMGMS1G80J1001278| 20181206|                     0.0|110.98333333333332|                        0.0|
|LMGMS1G80J1001278| 20181220|      125.96666666666667|1467.8166666666668|        0.08581907368086386|
+-----------------+---------+------------------------+------------------+---------------------------+
only showing top 2 rows



In [56]:
cols = [when(~col(x).isin("NULL", "NA", "NaN",""), col(x)).alias(x) for x in df_nd.columns]
df_nd = df_nd.select(*cols)
df_nd.registerTempTable('night_driving')
sql_cmd = """CREATE TABLE ubi.temp_nd AS SELECT vin,start_day, daily_night_driving_mins, daily_driving_mins,  
            percent_daily_night_driving FROM night_driving"""
print(sql_cmd)
hc.sql(sql_cmd)
print('Table temp_nd creation done.')

CREATE TABLE ubi.temp_nd AS SELECT vin,start_day, daily_night_driving_mins, daily_driving_mins,  
            percent_daily_night_driving FROM night_driving


AnalysisException: u'`ubi`.`temp_nd` already exists.;'

# PM Peak

In [57]:
query = """select vin, start_time, start_day, end_time, 
   from_unixtime(unix_timestamp(CONCAT(start_day, ' ', '16:00:00'), 'yyyyMMdd HH:mm:ss')) as peak_start,
   from_unixtime(unix_timestamp(CONCAT(start_day, ' ', '19:00:00'), 'yyyyMMdd HH:mm:ss')) as peak_end 
   from ubi.conv_trips_complete"""    

In [58]:
df_a = hive2spark(hc, query)

In [59]:
new_column_1=F.when(df_a.start_time > df_a.peak_start,df_a.start_time.astype('Timestamp'))\
.otherwise(df_a.peak_start.astype('Timestamp'))
new_column_2=F.when(df_a.end_time < df_a.peak_end,df_a.end_time.astype('Timestamp'))\
.otherwise(df_a.peak_end.astype('Timestamp'))
df_a=df_a.withColumn("peak_start_time",new_column_1)
df_b=df_a.withColumn("peak_end_time",new_column_2)

In [60]:
df_b.show(5)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|    peak_start_time|      peak_end_time|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 16:00:00|2018-06-29 19:00:00|2018-06-29 16:00:00|2018-06-29 06:47:22|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 16:00:00|2018-07-07 19:00:00|2018-07-07 16:00:00|2018-07-07 01:19:43|
|LMGMS1G80J1009719|2018-08-14 02:18:32| 20180814|2018-08-14 02:24:00|2018-08-14 16:00:00|2018-08-14 19:00:00|2018-08-14 16:00:00|2018-08-14 02:24:00|
|LMGMS1G80J1015701|2019-01-12 01:15:52| 20190112|2019-01-12 01:39:32|2019-01-12 16:00:00|2019-01-12 

In [61]:
new_column_3 = (to_timestamp(df_b.end_time.astype('Timestamp'),normaltimeFormat).cast("long")
                -to_timestamp(df_b.start_time.astype('Timestamp'),normaltimeFormat).cast("long"))/60
df_b = df_b.withColumn("driving_mins",new_column_3)
df_b.show(2)

+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|              vin|         start_time|start_day|           end_time|         peak_start|           peak_end|    peak_start_time|      peak_end_time|      driving_mins|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|LMGMS1G80H1000254|2018-06-29 06:37:51| 20180629|2018-06-29 06:47:22|2018-06-29 16:00:00|2018-06-29 19:00:00|2018-06-29 16:00:00|2018-06-29 06:47:22| 9.516666666666667|
|LMGMS1G80J1007761|2018-07-07 00:59:44| 20180707|2018-07-07 01:19:43|2018-07-07 16:00:00|2018-07-07 19:00:00|2018-07-07 16:00:00|2018-07-07 01:19:43|19.983333333333334|
+-----------------+-------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+------

In [62]:
new_column_4=F.when(df_b.peak_end_time > df_b.peak_start_time,(to_timestamp(df_b.peak_end_time,normaltimeFormat).cast("long")
                                   -to_timestamp(df_b.peak_start_time,normaltimeFormat).cast("long"))/60)\
.otherwise(0)
df_c=df_b.withColumn("peak_driving_mins",new_column_4)

In [63]:
df_c=df_c.groupBy("vin","start_day").agg(expr("sum(peak_driving_mins)").alias("daily_peak_driving_mins"),
                                  expr("sum(driving_mins)").alias("daily_driving_mins"))

In [64]:
df_d=df_c.withColumn("thirty_day_after", date_add(to_timestamp(df_c.start_day,"yyyyMMdd"),30))

In [65]:
df_d.show(2)

+-----------------+---------+-----------------------+------------------+----------------+
|              vin|start_day|daily_peak_driving_mins|daily_driving_mins|thirty_day_after|
+-----------------+---------+-----------------------+------------------+----------------+
|LMGMS1G82J1006787| 20180914|                    0.0|168.41666666666666|      2018-10-14|
|LMGMS1G84J1015121| 20181018|                    0.0|             383.2|      2018-11-17|
+-----------------+---------+-----------------------+------------------+----------------+
only showing top 2 rows



In [66]:
df_f = df2.alias("e").join(df_d.alias("d"), (df_d.vin == df2.vin) & (df2.start_day <= df_d.thirty_day_after) & (df2.start_day > df_d.start_day))\
.select("e.vin","e.start_day","d.daily_driving_mins","d.daily_peak_driving_mins") 
df_f.show(5)

+-----------------+---------+------------------+-----------------------+
|              vin|start_day|daily_driving_mins|daily_peak_driving_mins|
+-----------------+---------+------------------+-----------------------+
|LMGMS1G80J1001278| 20181212| 4.283333333333333|                    0.0|
|LMGMS1G80J1001278| 20181212| 95.60000000000001|                    0.0|
|LMGMS1G80J1001278| 20181212|             209.8|                    0.0|
|LMGMS1G80J1001278| 20181212| 82.93333333333334|                    0.0|
|LMGMS1G80J1001278| 20181212|237.76666666666668|                    0.0|
+-----------------+---------+------------------+-----------------------+
only showing top 5 rows



In [67]:
df_pm=df_f.groupBy("vin","start_day").agg(expr("sum(daily_peak_driving_mins)").alias("daily_peak_driving_mins"),
                                  expr("sum(daily_driving_mins)").alias("daily_driving_mins"))
df_pm.withColumn("percent_daily_peak_driving", (F.col("daily_peak_driving_mins") / F.col("daily_driving_mins")))

DataFrame[vin: string, start_day: string, daily_peak_driving_mins: double, daily_driving_mins: double, percent_daily_peak_driving: double]

In [68]:
cols = [when(~col(x).isin("NULL", "NA", "NaN",""), col(x)).alias(x) for x in df_pm.columns]
df_pm = df_pm.select(*cols)
df_pm.registerTempTable('pm_peak')
sql_cmd = """CREATE TABLE ubi.temp_pm AS SELECT vin,start_day, daily_peak_driving_mins, daily_driving_mins,  
            percent_daily_peak_driving FROM pm_peak"""
print(sql_cmd)
hc.sql(sql_cmd)
print('Table temp_pm creation done.')

CREATE TABLE ubi.temp_pm AS SELECT vin,start_day, daily_peak_driving_mins, daily_driving_mins,  
            percent_daily_peak_driving FROM pm_peak


AnalysisException: u"Invalid call to dataType on unresolved object, tree: 'percent_daily_peak_driving"

# entropy

In [88]:
query = """select vin, count(*) as totalfrequency from ubi.conv_trips_complete group by vin""" 
df_f = hive2spark(hc, query)

In [89]:
df_trip = hive2spark(hc, "select * from ubi.conv_trips_complete")

In [91]:
df_f = df_trip.alias("e").join(df_f.alias("f"), df_trip.vin == df_f.vin)\
.select("e.vin","e.start_loc_lat","e.start_loc_lon","e.end_loc_lat","end_loc_lon","f.totalfrequency") 
df_f.show(5)

+-----------------+------------------+------------------+------------------+------------------+--------------+
|              vin|     start_loc_lat|     start_loc_lon|       end_loc_lat|       end_loc_lon|totalfrequency|
+-----------------+------------------+------------------+------------------+------------------+--------------+
|LMGMS1G80J1001278|22.973869444444443|114.70871944444444|22.975155555555553|114.70485833333333|           997|
|LMGMS1G80J1001278|22.977430555555554|114.71240833333334|22.975330555555555|114.70467222222223|           997|
|LMGMS1G80J1001278|22.974494444444442|114.70517222222223|22.969052777777776|114.70706666666666|           997|
|LMGMS1G80J1001278|22.973863888888886|114.70868333333334|22.974852777777777|          114.7047|           997|
|LMGMS1G80J1001278| 22.97539722222222|114.70496666666666|23.086427777777775|114.47661111111111|           997|
+-----------------+------------------+------------------+------------------+------------------+--------------+
o

In [92]:
#df4 = df_f.select(F.round(F.col("start_lon_lat").cast("float"),3).alias("start_loc_lat"))
df_f = df_f.withColumn("start_loc_lat", round("start_loc_lat", 3))
df_f.show(4)

+-----------------+-------------+------------------+------------------+------------------+--------------+
|              vin|start_loc_lat|     start_loc_lon|       end_loc_lat|       end_loc_lon|totalfrequency|
+-----------------+-------------+------------------+------------------+------------------+--------------+
|LMGMS1G80J1001278|       22.468|112.75563888888888|22.363736111111113|112.68298611111112|           997|
|LMGMS1G80J1001278|       22.591|114.14512500000001|22.974899999999998| 114.7047388888889|           997|
|LMGMS1G80J1001278|       22.993|114.71179166666667|22.974922222222222|114.70473611111112|           997|
|LMGMS1G80J1001278|       22.601|114.04439444444444|22.567680555555555|114.05658888888888|           997|
+-----------------+-------------+------------------+------------------+------------------+--------------+
only showing top 4 rows



In [93]:
df_f = df_f.withColumn("start_loc_lon", round("start_loc_lat", 3))
df_f = df_f.withColumn("end_loc_lat", round("end_loc_lat", 3))
df_f = df_f.withColumn("end_loc_lon", round("end_loc_lon", 3))

In [94]:
df_f.show(5)

+-----------------+-------------+-------------+-----------+-----------+--------------+
|              vin|start_loc_lat|start_loc_lon|end_loc_lat|end_loc_lon|totalfrequency|
+-----------------+-------------+-------------+-----------+-----------+--------------+
|LMGMS1G80J1001278|       22.977|       22.977|     22.975|    114.705|           997|
|LMGMS1G80J1001278|       22.974|       22.974|     22.969|    114.707|           997|
|LMGMS1G80J1001278|       22.974|       22.974|     22.975|    114.705|           997|
|LMGMS1G80J1001278|        22.98|        22.98|     22.975|    114.705|           997|
|LMGMS1G80J1001278|       22.578|       22.578|     22.557|    114.128|           997|
+-----------------+-------------+-------------+-----------+-----------+--------------+
only showing top 5 rows



In [95]:
df_f = df_f.withColumn("probability",  (1 / F.col("totalfrequency")))

In [96]:
df_f.show(5)

+-----------------+-------------+-------------+-----------+-----------+--------------+--------------------+
|              vin|start_loc_lat|start_loc_lon|end_loc_lat|end_loc_lon|totalfrequency|         probability|
+-----------------+-------------+-------------+-----------+-----------+--------------+--------------------+
|LMGMS1G80J1001278|       22.578|       22.578|     22.568|    114.056|           997|0.001003009027081...|
|LMGMS1G80J1001278|        22.98|        22.98|     22.975|    114.705|           997|0.001003009027081...|
|LMGMS1G80J1001278|       22.969|       22.969|     22.972|    114.711|           997|0.001003009027081...|
|LMGMS1G80J1001278|       23.085|       23.085|     23.087|    114.479|           997|0.001003009027081...|
|LMGMS1G80J1001278|       22.954|       22.954|     22.975|    114.705|           997|0.001003009027081...|
+-----------------+-------------+-------------+-----------+-----------+--------------+--------------------+
only showing top 5 rows



In [101]:
df_f = df_f.drop(F.col("totalfrequency"))
df_f.show(2)

+-----------------+-------------+-------------+-----------+-----------+--------------------+
|              vin|start_loc_lat|start_loc_lon|end_loc_lat|end_loc_lon|         probability|
+-----------------+-------------+-------------+-----------+-----------+--------------------+
|LMGMS1G80J1001278|       22.578|       22.578|     22.568|    114.056|0.001003009027081...|
|LMGMS1G80J1001278|       22.974|       22.974|     22.975|    114.705|0.001003009027081...|
+-----------------+-------------+-------------+-----------+-----------+--------------------+
only showing top 2 rows



In [103]:
df_g = df_f.groupBy("vin","start_loc_lat","start_loc_lon", "end_loc_lat", "end_loc_lon")\
            .agg(expr("sum(probability)").alias("probability"))
df_g.show(5)

+-----------------+-------------+-------------+-----------+-----------+--------------------+
|              vin|start_loc_lat|start_loc_lon|end_loc_lat|end_loc_lon|         probability|
+-----------------+-------------+-------------+-----------+-----------+--------------------+
|LMGMS1G80J1001278|       22.468|       22.468|     22.364|    112.683|0.001003009027081...|
|LMGMS1G80J1001278|       22.591|       22.591|     22.975|    114.705|0.003009027081243...|
|LMGMS1G80J1001278|       22.975|       22.975|     23.086|    114.477|0.002006018054162...|
|LMGMS1G80J1001278|       22.988|       22.988|     22.975|    114.705|0.007021063189568...|
|LMGMS1G80J1001278|       22.601|       22.601|     22.568|    114.057|0.001003009027081...|
+-----------------+-------------+-------------+-----------+-----------+--------------------+
only showing top 5 rows



In [105]:
df_h = df_g.groupBy("vin").agg(expr("sum(-probability * log2(probability))").alias("entropy_all"))
df_h.show(5)

+-----------------+-----------------+
|              vin|      entropy_all|
+-----------------+-----------------+
|LMGMS1G80J1001278| 8.83199050068945|
|LMGMS1G80J1004780|7.954114910693735|
|LMGMS1G80J1014239|6.336553572571366|
|LMGMS1G80J1014600| 8.73289802020143|
|LMGMS1G80J1014726|9.339963602427142|
+-----------------+-----------------+
only showing top 5 rows

